In [1]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd
import dill

In [9]:
import json

In [18]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 2AQwxtl9lZmWOYwwGv3dptsb2GJ_21ZJCVJ59W9RjFNtN2CJ6
!./ngrok http 80

--2022-07-22 14:48:05--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz.3’

ngrok-stable-linux- 100%[===================>]  13.13M  8.41MB/s    in 1.6s    

2022-07-22 14:48:06 (8.41 MB/s) - ‘ngrok-stable-linux-amd64.tgz.3’ saved [13770165/13770165]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
/content/ngrok-stable-linux-amd64.tgz.3


### Создаем сервис для обработки запросов к модели

In [19]:
# Загружаем обученные модели
with open('gbc_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)
  

In [20]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

Запустить сервис и не глушить его, пока работаем

In [21]:
# Обработчики и запуск Flask
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    comment = ""
    request_json = request.get_json()
    
    if request_json["comment"]:
        comment = request_json['comment']
    
    print(comment)  
    preds = model.predict_proba(pd.DataFrame({"comment": [comment]}))
    data["predictions"] = preds[:, 1][0]
    data["comment"] = comment
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9061-35-196-60-181.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/Jul/2022 14:51:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2022 14:51:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2022 14:51:20] "GET /predict HTTP/1.1" 405 -
127.0.0.1 - - [22/Jul/2022 14:51:38] "GET / HTTP/1.1" 200 -
[2022-07-22 15:06:29,627] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  Fi

['Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them.\xa0As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity. \xa0Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.', 'ustwo offers you the opportunity to be yourself, whilst delivering

127.0.0.1 - - [22/Jul/2022 15:13:36] "POST /predict HTTP/1.1" 200 -


Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them. As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity.  Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.
OK
